In [1]:
import pandas as pd
import numpy as np
import MeCab

from tqdm import tqdm_notebook
import time

In [2]:
%%time
# read original data
pd.set_option('display.max_colwidth', -1)
csvData = pd.read_csv("rental.csv", delimiter=",")
display(csvData)

,shop_url,shop_id,item_id,item_name,item_url,catch_copy,pc_caption,caption,tag_id2,tag_id3,gn1,Category
0,e-kimono-rental,301051,10002322,【レンタル】振袖レンタル（2月〜12月）〔6S713 白地〕〔結婚式〕〔成人式〕〔結納〕〔卒業式〕【レンタル着物】【振袖】〔貸衣装〕【フルセット】【小物一式つき】〔振り袖〕【往復送料無料】【新品足袋プレゼント】★,http://item.rakuten.co.jp/e-kimono-rental/6s713-1,白地に目の冴える赤ぼかし。およばれ パーティーなどにもアレンジを効かせて！,※カレンダーの表示に少しお時間がかかります。表示までしばらくお待ちください。×印のところは申し訳ございませんが既にご予約済みとなっております。ご了承下さい。カレンダーの予約状況が反映されるまで時間がかかることがありますので、予約確定でない場合がございます。ショップからのメールにてご確認をお願い致します。,モデル身長：163cm ※商品写真は撮影状況、お客様の表示環境等により実物と印象の異なる場合がございます。あらかじめご了承ください。 ※画像の小物類は、撮影用の為お付けする物と異なります ■付属品 振袖／袋帯／長襦袢（白半衿付き）／帯あげ／帯締め／草履／バッグ／肌着（ワンピースタイプ）／腰ひも 4本／和装ベルト／前板／後板／伊逹締め 2枚／帯枕／三重仮紐／重ね衿／衿芯／足袋（新品プレゼント） 振袖レンタル（2月〜12月）〔結婚式〕〔成人式〕〔結納〕〔卒業式〕【レンタル着物】【振袖】〔貸衣装〕【フルセット】【小物一式つき】〔振り袖〕【往復送料無料】【新品足袋プレゼント】★〔6s713 白地〕 白地に目の冴える赤ぼかし。およばれ パーティーなどにもアレンジを効かせて！ ■着物対応可能身長・サイズ 163cm位まで 13号まで ■色/柄：白地 赤薔薇ぼかし ■生地 着物：正絹 【ご用意して頂く物】 補正用タオル ※念の為、着付けされる美容院様にご確認ください。,NaN,5002023,レディースファッション,Rental
1,oharibako,206033,10002303,【店内全品ポイント10倍！11/5 23:59まで】七五三 着物 レンタル 7歳 フルセット アンティーク 正絹 【七歳】おはりばこの七五三レンタル衣裳♪正絹アンティーク着物で七五三！！【No.n07】【花輪乱菊】【桃色紋錦紗地花丸紋尽四つ身】【巾着＆はこせこ付き】fy16REN07,http://item.rakuten.co.jp/oharibako/n07,おはりばこの七五三レンタル衣裳・レンタル衣装♪正絹アンティーク着物で七五三！,NaN,"&nbsp; 桃色紋錦紗地花丸紋尽四つ身 &nbsp; 鮮やかな桃色の地色に、菊がたくさん描かれた大きな丸紋が染められています。 勢いのあるリアルな乱菊と、文様化されたねじ梅の対比が面白いですね。 地色はとってもガーリーなピンク色。差し色も朱色や赤など、女の子が大好きなお色ばかり♪ 着るだけで昔のお嬢様になれそうな、かわいいお着物です。 ※帯まわりのコーディネイト（帯、帯揚げ、帯締め、しごき）について、 予約状況により、画像のものと異なる場合がございますが、 あらかじめご了承ください。 &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; 桃色紋錦紗地花丸紋尽四つ身 商品情報 サイズ 身丈:約122cm 裄丈:約51cm 適応身長:105〜117cm ※肩上げをした状態でお送りします。腰上げはしておりません。 ※アンティーク着物のため、現代のお着物とは仕立て・サイズが異なる場合がございます。 着物とお子様の裄丈の寸法を必ずご確認ください。&gt;&gt;&gt;【裄丈の測り方はこちら】 素材 表地:正絹 裏地:正絹 製造 国産 戦前のアンティーク着物 配送方法 宅急便（ゆうパック） 注意事項 商品代金の支払は、カード決済のみとなります。 銀行振込・代金引換はご利用いただけませんのご了承ください。 &nbsp; ■「安心保険」ついて 七五三の晴れの日の衣裳をご検討頂きありがとうございます。 おはりばこでは、商品代金の5％の料金で保険が掛けられます！！ レンタルのご利用規約にも記載させて頂いておりますように、損傷し修復が必要となる場合は修復代金に相当する金額を弁償して頂くこととなっております。 例えば・・・ ・車の汚れが着物についてしまった。 ・子供が着物の裾を踏んで汚してしまった。 ・子供用の扇子を開け閉めして壊してしまった。（等 そんなときに安心できるのが、『安心保険サービス』でございます。 お衣裳の1セットにつき、レンタル代金の5％のご料金をお支払い頂けましたら、修理代金は一切発生致しません。 安心して当日をお過ごし頂きます為に、レンタルご利用のお客様すべてに、ご加入をお勧めさせて頂いております。 ■安心保険適用外 事項 下記の事項が発生致しました場合には、保険適用外となります。 ※使用不可能となる汚れ、破れ等の破損した場合。 ※飲食による汚れ。 ※タバコの焼け焦げ。 ※レンタル商品を紛失された場合。 （等 &nbsp; ■レンタルお申込み期間について ご注文日より最短10日以降、最長約半年までの期間内のみ受付いたしております。 ■レンタルには、当店での髪飾りまたは七五三商品の購入が必要です。 レンタル着物をご購入とご一緒に当店おはりばこの髪飾りまたは七五三商品を 5,000円以上ご購入くださいませ。 ※レンタル着物ご注文時点より、 過去半年以内に当店の髪飾りをご購入のお客様も対象となり、レンタルが可能です。 レンタル着物購入後に、髪飾りを選んで購入希望のお客様は、 レンタル利用日までご注文くださいませ。 髪飾りについてコーディネイト・ご相談などはお気軽にご連絡くださいませ。 ■髪飾りをご検討中のお客様へ 毎年10月頃から完売商品も出てまいりますので、髪飾りをすでに決めている場合は、先に楽天市場店または、実店舗、催事出店などでご購入いただくことをおすすめいたします。 また、七五三髪飾りは5月末〜11月末までの製作となっております。前撮り・後撮りの場合、在庫がない場合もございますが、ご了承ください。 ■おはりばこの七歳用の帯は作り帯ではありません！ 当店の七歳用の帯は、大人用の高級丸帯を子供用に仕立て直した帯です。 振袖の着付けなどと同じように、帯結びのできる着付師さんが必要です。 必ず写真館や美容室にご確認の上、ご利用ください。 ※三歳用帯コーデの帯は作り帯です。 ■アンティーク着物には、時代の汚れや傷みがあります。 おはりばこが扱う七五三衣裳は、全て戦前に作られたアンティーク着物です。 70年以上前に作られたお着物ですので、 時代相応の汚れ・やけ・色あせ・すれ・傷みが必ずあります。 もちろん、晴れ着として着用いただけるレベルのものばかりを取り揃えておりますが、 完全な状態のものは一着もありません。 汚れなどが気になる方は、ご利用をご遠慮いただくようおねがいいたします。 ■着用しての外出・お詣りについて 7月、8月のご利用に関して、汗じみ防止のため、着用しての外出やお詣りは不可となっています。室内での撮影のみでのご利用をお願いいたします。 その他の時期に関しては、室内での撮影はもちろん外出・お詣りは可能でございます。 &nbsp; &nbsp; &nbsp;",NaN,5002023,キッズ・ベビー・マタニティ,Rental
2,rentalmiyuki,279558,10000679,【レンタル】【留袖 レンタル】レンタル留袖 mito_4 北256 グリーン のし 正絹|母親 黒留袖 フルセット マイセレクト 留袖レンタル レンタル着物 草履バッグセット 黒留袖レンタル 柄 結婚式 着物セット 黒留 帯締め 黒留め袖 貸衣装 お宮参り おすすめ レディース,http://item.rakuten.co.jp/rentalmiyuki/mito_4,厳選された留袖 対応身長 155cmから165cm位 洋服号数 7〜13号(ヒップサイズ97cm以下),&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;,&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 当店ではアップグレードの表地：織りの日本製ブランド草履バッグがございます。↓ 商品詳細 【商品番号】 mito_4：北256 グリーン のし鶸色のぼかしで鬘帯文に洋花を染め抜き大輪の菊を金彩でアクセントにしたモダン感覚の留袖です。 【素材】 表地：正絹100% 裏地：比翼仕立合成繊維 【サイズ】 裄70cm・着丈165cm・袖丈49cm サイズ表は*こちらをクリック* してください。 【対応身長】 155cmから165cm位 ／ 洋服サイズ 7〜13号位まで 【注意事項】 ※商品の色目につきましては、お客様のご覧になられますパソコン機器及びモニタなどの違いにより実際の商品素材の色と相違する場合が御座います。予めご了承下さい。 【注意事項2】 ※帯に関しましては、商品の出荷状況などにより同じ物をご用意させて頂くお約束ができません。

CPU times: user 5.2 s, sys: 634 ms, total: 5.83 s
Wall time: 6.52 s


In [3]:
%%time
# extract text columns
txtCols = ['item_name','catch_copy','pc_caption','caption']
dataTxt = csvData[txtCols + ['Category']]

display(dataTxt)

,item_name,catch_copy,pc_caption,caption,Category
0,【レンタル】振袖レンタル（2月〜12月）〔6S713 白地〕〔結婚式〕〔成人式〕〔結納〕〔卒業式〕【レンタル着物】【振袖】〔貸衣装〕【フルセット】【小物一式つき】〔振り袖〕【往復送料無料】【新品足袋プレゼント】★,白地に目の冴える赤ぼかし。およばれ パーティーなどにもアレンジを効かせて！,※カレンダーの表示に少しお時間がかかります。表示までしばらくお待ちください。×印のところは申し訳ございませんが既にご予約済みとなっております。ご了承下さい。カレンダーの予約状況が反映されるまで時間がかかることがありますので、予約確定でない場合がございます。ショップからのメールにてご確認をお願い致します。,モデル身長：163cm ※商品写真は撮影状況、お客様の表示環境等により実物と印象の異なる場合がございます。あらかじめご了承ください。 ※画像の小物類は、撮影用の為お付けする物と異なります ■付属品 振袖／袋帯／長襦袢（白半衿付き）／帯あげ／帯締め／草履／バッグ／肌着（ワンピースタイプ）／腰ひも 4本／和装ベルト／前板／後板／伊逹締め 2枚／帯枕／三重仮紐／重ね衿／衿芯／足袋（新品プレゼント） 振袖レンタル（2月〜12月）〔結婚式〕〔成人式〕〔結納〕〔卒業式〕【レンタル着物】【振袖】〔貸衣装〕【フルセット】【小物一式つき】〔振り袖〕【往復送料無料】【新品足袋プレゼント】★〔6s713 白地〕 白地に目の冴える赤ぼかし。およばれ パーティーなどにもアレンジを効かせて！ ■着物対応可能身長・サイズ 163cm位まで 13号まで ■色/柄：白地 赤薔薇ぼかし ■生地 着物：正絹 【ご用意して頂く物】 補正用タオル ※念の為、着付けされる美容院様にご確認ください。,Rental
1,【店内全品ポイント10倍！11/5 23:59まで】七五三 着物 レンタル 7歳 フルセット アンティーク 正絹 【七歳】おはりばこの七五三レンタル衣裳♪正絹アンティーク着物で七五三！！【No.n07】【花輪乱菊】【桃色紋錦紗地花丸紋尽四つ身】【巾着＆はこせこ付き】fy16REN07,おはりばこの七五三レンタル衣裳・レンタル衣装♪正絹アンティーク着物で七五三！,NaN,"&nbsp; 桃色紋錦紗地花丸紋尽四つ身 &nbsp; 鮮やかな桃色の地色に、菊がたくさん描かれた大きな丸紋が染められています。 勢いのあるリアルな乱菊と、文様化されたねじ梅の対比が面白いですね。 地色はとってもガーリーなピンク色。差し色も朱色や赤など、女の子が大好きなお色ばかり♪ 着るだけで昔のお嬢様になれそうな、かわいいお着物です。 ※帯まわりのコーディネイト（帯、帯揚げ、帯締め、しごき）について、 予約状況により、画像のものと異なる場合がございますが、 あらかじめご了承ください。 &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; 桃色紋錦紗地花丸紋尽四つ身 商品情報 サイズ 身丈:約122cm 裄丈:約51cm 適応身長:105〜117cm ※肩上げをした状態でお送りします。腰上げはしておりません。 ※アンティーク着物のため、現代のお着物とは仕立て・サイズが異なる場合がございます。 着物とお子様の裄丈の寸法を必ずご確認ください。&gt;&gt;&gt;【裄丈の測り方はこちら】 素材 表地:正絹 裏地:正絹 製造 国産 戦前のアンティーク着物 配送方法 宅急便（ゆうパック） 注意事項 商品代金の支払は、カード決済のみとなります。 銀行振込・代金引換はご利用いただけませんのご了承ください。 &nbsp; ■「安心保険」ついて 七五三の晴れの日の衣裳をご検討頂きありがとうございます。 おはりばこでは、商品代金の5％の料金で保険が掛けられます！！ レンタルのご利用規約にも記載させて頂いておりますように、損傷し修復が必要となる場合は修復代金に相当する金額を弁償して頂くこととなっております。 例えば・・・ ・車の汚れが着物についてしまった。 ・子供が着物の裾を踏んで汚してしまった。 ・子供用の扇子を開け閉めして壊してしまった。（等 そんなときに安心できるのが、『安心保険サービス』でございます。 お衣裳の1セットにつき、レンタル代金の5％のご料金をお支払い頂けましたら、修理代金は一切発生致しません。 安心して当日をお過ごし頂きます為に、レンタルご利用のお客様すべてに、ご加入をお勧めさせて頂いております。 ■安心保険適用外 事項 下記の事項が発生致しました場合には、保険適用外となります。 ※使用不可能となる汚れ、破れ等の破損した場合。 ※飲食による汚れ。 ※タバコの焼け焦げ。 ※レンタル商品を紛失された場合。 （等 &nbsp; ■レンタルお申込み期間について ご注文日より最短10日以降、最長約半年までの期間内のみ受付いたしております。 ■レンタルには、当店での髪飾りまたは七五三商品の購入が必要です。 レンタル着物をご購入とご一緒に当店おはりばこの髪飾りまたは七五三商品を 5,000円以上ご購入くださいませ。 ※レンタル着物ご注文時点より、 過去半年以内に当店の髪飾りをご購入のお客様も対象となり、レンタルが可能です。 レンタル着物購入後に、髪飾りを選んで購入希望のお客様は、 レンタル利用日までご注文くださいませ。 髪飾りについてコーディネイト・ご相談などはお気軽にご連絡くださいませ。 ■髪飾りをご検討中のお客様へ 毎年10月頃から完売商品も出てまいりますので、髪飾りをすでに決めている場合は、先に楽天市場店または、実店舗、催事出店などでご購入いただくことをおすすめいたします。 また、七五三髪飾りは5月末〜11月末までの製作となっております。前撮り・後撮りの場合、在庫がない場合もございますが、ご了承ください。 ■おはりばこの七歳用の帯は作り帯ではありません！ 当店の七歳用の帯は、大人用の高級丸帯を子供用に仕立て直した帯です。 振袖の着付けなどと同じように、帯結びのできる着付師さんが必要です。 必ず写真館や美容室にご確認の上、ご利用ください。 ※三歳用帯コーデの帯は作り帯です。 ■アンティーク着物には、時代の汚れや傷みがあります。 おはりばこが扱う七五三衣裳は、全て戦前に作られたアンティーク着物です。 70年以上前に作られたお着物ですので、 時代相応の汚れ・やけ・色あせ・すれ・傷みが必ずあります。 もちろん、晴れ着として着用いただけるレベルのものばかりを取り揃えておりますが、 完全な状態のものは一着もありません。 汚れなどが気になる方は、ご利用をご遠慮いただくようおねがいいたします。 ■着用しての外出・お詣りについて 7月、8月のご利用に関して、汗じみ防止のため、着用しての外出やお詣りは不可となっています。室内での撮影のみでのご利用をお願いいたします。 その他の時期に関しては、室内での撮影はもちろん外出・お詣りは可能でございます。 &nbsp; &nbsp; &nbsp;",Rental
2,【レンタル】【留袖 レンタル】レンタル留袖 mito_4 北256 グリーン のし 正絹|母親 黒留袖 フルセット マイセレクト 留袖レンタル レンタル着物 草履バッグセット 黒留袖レンタル 柄 結婚式 着物セット 黒留 帯締め 黒留め袖 貸衣装 お宮参り おすすめ レディース,厳選された留袖 対応身長 155cmから165cm位 洋服号数 7〜13号(ヒップサイズ97cm以下),&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;,&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 当店ではアップグレードの表地：織りの日本製ブランド草履バッグがございます。↓ 商品詳細 【商品番号】 mito_4：北256 グリーン のし鶸色のぼかしで鬘帯文に洋花を染め抜き大輪の菊を金彩でアクセントにしたモダン感覚の留袖です。 【素材】 表地：正絹100% 裏地：比翼仕立合成繊維 【サイズ】 裄70cm・着丈165cm・袖丈49cm サイズ表は*こちらをクリック* してください。 【対応身長】 155cmから165cm位 ／ 洋服サイズ 7〜13号位まで 【注意事項】 ※商品の色目につきましては、お客様のご覧になられますパソコン機器及びモニタなどの違いにより実際の商品素材の色と相違する場合が御座います。予めご了承下さい。 【注意事項2】 ※帯に関しましては、商品の出荷状況などにより同じ物をご用意させて頂くお約束ができません。同じような帯のイメージでコーディネートさせて頂きますので、ご了承の程、よろしくお願い致します。 【コーディネート】 帯柄、帯揚、帯〆、伊達衿、半衿、草履、バックは当店プロスタッフがお着物にあったものをコーディネートさせて頂きます。 ご要望などのあるお客様は備考欄に具体的にご記入ください。,Rental
3,【レンタル】[東京礼服センター発送商品][フルセット]喪服 レンタル 礼服 レンタル〔K032〕桂由美 レディース ブラックフォーマル アンサンブル ワンピース レンタル[お通夜][お葬式]{9号}{11号}[女性 礼服][レディース]【fy16REN07】,「礼装バッグ」「靴」「ネックレス」すべてそろったフルセットレンタル 「礼装用ストッキン

CPU times: user 92.6 ms, sys: 10.4 ms, total: 103 ms
Wall time: 132 ms


In [4]:
%%time
# remove irrelavant words
dataTxtCopy = dataTxt.copy()
dataTxtCopy.replace(['&nbsp;'],'',regex=True,inplace=True)
dataTxtCopy.replace({'0':'', np.nan:''},inplace=True)
dataTxtCopy.applymap(lambda x: x.rstrip() if type(x) is str else x)

display(dataTxtCopy)

,item_name,catch_copy,pc_caption,caption,Category
0,【レンタル】振袖レンタル（2月〜12月）〔6S713 白地〕〔結婚式〕〔成人式〕〔結納〕〔卒業式〕【レンタル着物】【振袖】〔貸衣装〕【フルセット】【小物一式つき】〔振り袖〕【往復送料無料】【新品足袋プレゼント】★,白地に目の冴える赤ぼかし。およばれ パーティーなどにもアレンジを効かせて！,※カレンダーの表示に少しお時間がかかります。表示までしばらくお待ちください。×印のところは申し訳ございませんが既にご予約済みとなっております。ご了承下さい。カレンダーの予約状況が反映されるまで時間がかかることがありますので、予約確定でない場合がございます。ショップからのメールにてご確認をお願い致します。,モデル身長：163cm ※商品写真は撮影状況、お客様の表示環境等により実物と印象の異なる場合がございます。あらかじめご了承ください。 ※画像の小物類は、撮影用の為お付けする物と異なります ■付属品 振袖／袋帯／長襦袢（白半衿付き）／帯あげ／帯締め／草履／バッグ／肌着（ワンピースタイプ）／腰ひも 4本／和装ベルト／前板／後板／伊逹締め 2枚／帯枕／三重仮紐／重ね衿／衿芯／足袋（新品プレゼント） 振袖レンタル（2月〜12月）〔結婚式〕〔成人式〕〔結納〕〔卒業式〕【レンタル着物】【振袖】〔貸衣装〕【フルセット】【小物一式つき】〔振り袖〕【往復送料無料】【新品足袋プレゼント】★〔6s713 白地〕 白地に目の冴える赤ぼかし。およばれ パーティーなどにもアレンジを効かせて！ ■着物対応可能身長・サイズ 163cm位まで 13号まで ■色/柄：白地 赤薔薇ぼかし ■生地 着物：正絹 【ご用意して頂く物】 補正用タオル ※念の為、着付けされる美容院様にご確認ください。,Rental
1,【店内全品ポイント10倍！11/5 23:59まで】七五三 着物 レンタル 7歳 フルセット アンティーク 正絹 【七歳】おはりばこの七五三レンタル衣裳♪正絹アンティーク着物で七五三！！【No.n07】【花輪乱菊】【桃色紋錦紗地花丸紋尽四つ身】【巾着＆はこせこ付き】fy16REN07,おはりばこの七五三レンタル衣裳・レンタル衣装♪正絹アンティーク着物で七五三！,,"桃色紋錦紗地花丸紋尽四つ身 鮮やかな桃色の地色に、菊がたくさん描かれた大きな丸紋が染められています。 勢いのあるリアルな乱菊と、文様化されたねじ梅の対比が面白いですね。 地色はとってもガーリーなピンク色。差し色も朱色や赤など、女の子が大好きなお色ばかり♪ 着るだけで昔のお嬢様になれそうな、かわいいお着物です。 ※帯まわりのコーディネイト（帯、帯揚げ、帯締め、しごき）について、 予約状況により、画像のものと異なる場合がございますが、 あらかじめご了承ください。 桃色紋錦紗地花丸紋尽四つ身 商品情報 サイズ 身丈:約122cm 裄丈:約51cm 適応身長:105〜117cm ※肩上げをした状態でお送りします。腰上げはしておりません。 ※アンティーク着物のため、現代のお着物とは仕立て・サイズが異なる場合がございます。 着物とお子様の裄丈の寸法を必ずご確認ください。&gt;&gt;&gt;【裄丈の測り方はこちら】 素材 表地:正絹 裏地:正絹 製造 国産 戦前のアンティーク着物 配送方法 宅急便（ゆうパック） 注意事項 商品代金の支払は、カード決済のみとなります。 銀行振込・代金引換はご利用いただけませんのご了承ください。 ■「安心保険」ついて 七五三の晴れの日の衣裳をご検討頂きありがとうございます。 おはりばこでは、商品代金の5％の料金で保険が掛けられます！！ レンタルのご利用規約にも記載させて頂いておりますように、損傷し修復が必要となる場合は修復代金に相当する金額を弁償して頂くこととなっております。 例えば・・・ ・車の汚れが着物についてしまった。 ・子供が着物の裾を踏んで汚してしまった。 ・子供用の扇子を開け閉めして壊してしまった。（等 そんなときに安心できるのが、『安心保険サービス』でございます。 お衣裳の1セットにつき、レンタル代金の5％のご料金をお支払い頂けましたら、修理代金は一切発生致しません。 安心して当日をお過ごし頂きます為に、レンタルご利用のお客様すべてに、ご加入をお勧めさせて頂いております。 ■安心保険適用外 事項 下記の事項が発生致しました場合には、保険適用外となります。 ※使用不可能となる汚れ、破れ等の破損した場合。 ※飲食による汚れ。 ※タバコの焼け焦げ。 ※レンタル商品を紛失された場合。 （等 ■レンタルお申込み期間について ご注文日より最短10日以降、最長約半年までの期間内のみ受付いたしております。 ■レンタルには、当店での髪飾りまたは七五三商品の購入が必要です。 レンタル着物をご購入とご一緒に当店おはりばこの髪飾りまたは七五三商品を 5,000円以上ご購入くださいませ。 ※レンタル着物ご注文時点より、 過去半年以内に当店の髪飾りをご購入のお客様も対象となり、レンタルが可能です。 レンタル着物購入後に、髪飾りを選んで購入希望のお客様は、 レンタル利用日までご注文くださいませ。 髪飾りについてコーディネイト・ご相談などはお気軽にご連絡くださいませ。 ■髪飾りをご検討中のお客様へ 毎年10月頃から完売商品も出てまいりますので、髪飾りをすでに決めている場合は、先に楽天市場店または、実店舗、催事出店などでご購入いただくことをおすすめいたします。 また、七五三髪飾りは5月末〜11月末までの製作となっております。前撮り・後撮りの場合、在庫がない場合もございますが、ご了承ください。 ■おはりばこの七歳用の帯は作り帯ではありません！ 当店の七歳用の帯は、大人用の高級丸帯を子供用に仕立て直した帯です。 振袖の着付けなどと同じように、帯結びのできる着付師さんが必要です。 必ず写真館や美容室にご確認の上、ご利用ください。 ※三歳用帯コーデの帯は作り帯です。 ■アンティーク着物には、時代の汚れや傷みがあります。 おはりばこが扱う七五三衣裳は、全て戦前に作られたアンティーク着物です。 70年以上前に作られたお着物ですので、 時代相応の汚れ・やけ・色あせ・すれ・傷みが必ずあります。 もちろん、晴れ着として着用いただけるレベルのものばかりを取り揃えておりますが、 完全な状態のものは一着もありません。 汚れなどが気になる方は、ご利用をご遠慮いただくようおねがいいたします。 ■着用しての外出・お詣りについて 7月、8月のご利用に関して、汗じみ防止のため、着用しての外出やお詣りは不可となっています。室内での撮影のみでのご利用をお願いいたします。 その他の時期に関しては、室内での撮影はもちろん外出・お詣りは可能でございます。",Rental
2,【レンタル】【留袖 レンタル】レンタル留袖 mito_4 北256 グリーン のし 正絹|母親 黒留袖 フルセット マイセレクト 留袖レンタル レンタル着物 草履バッグセット 黒留袖レンタル 柄 結婚式 着物セット 黒留 帯締め 黒留め袖 貸衣装 お宮参り おすすめ レディース,厳選された留袖 対応身長 155cmから165cm位 洋服号数 7〜13号(ヒップサイズ97cm以下),,当店ではアップグレードの表地：織りの日本製ブランド草履バッグがございます。↓ 商品詳細 【商品番号】 mito_4：北256 グリーン のし鶸色のぼかしで鬘帯文に洋花を染め抜き大輪の菊を金彩でアクセントにしたモダン感覚の留袖です。 【素材】 表地：正絹100% 裏地：比翼仕立合成繊維 【サイズ】 裄70cm・着丈165cm・袖丈49cm サイズ表は*こちらをクリック* してください。 【対応身長】 155cmから165cm位 ／ 洋服サイズ 7〜13号位まで 【注意事項】 ※商品の色目につきましては、お客様のご覧になられますパソコン機器及びモニタなどの違いにより実際の商品素材の色と相違する場合が御座います。予めご了承下さい。 【注意事項2】 ※帯に関しましては、商品の出荷状況などにより同じ物をご用意させて頂くお約束ができません。同じような帯のイメージでコーディネートさせて頂きますので、ご了承の程、よろしくお願い致します。 【コーディネート】 帯柄、帯揚、帯〆、伊達衿、半衿、草履、バックは当店プロスタッフがお着物にあったものをコーディネートさせて頂きます。 ご要望などのあるお客様は備考欄に具体的にご記入ください。,Rental
3,【レンタル】[東京礼服センター発送商品][フルセット]喪服 レンタル 礼服 レンタル〔K032〕桂由美 レディース ブラックフォーマル アンサンブル ワンピース レンタル[お通夜][お葬式]{9号}{11号}[女性 礼服][レディース]【fy16REN07】,「礼装バッグ」「靴」「ネックレス」すべてそろったフルセットレンタル 「礼装用ストッキング」プレゼント♪ 喪服 礼服 レンタル クリーニング不要,"◆ご利用日の2日前にお届けします。 （ご注文いただいた日時によっては2日前にお届けできない場合がございます） ◆お急ぎの際はご利用日に間に合うようお届けします。 東京・大阪翌日午前中にお届けできます。 本州全域翌日のお届けが可能です（時間帯についてはお問い合わせください）。 ◆ご利用日の翌日に返却（発送）をお願いいたします。 ◆シーズン・土日祝日は注文が大変混みあいます。 お早目のご注文をお願いいたしま

CPU times: user 1.93 s, sys: 133 ms, total: 2.07 s
Wall time: 2.55 s


In [5]:
%%time
# combine text columns
allTxt = pd.DataFrame({'text': dataTxtCopy['item_name'] + dataTxtCopy['catch_copy'] + dataTxtCopy['pc_caption'] + dataTxtCopy['caption'], 'label': dataTxtCopy['Category']})
display(allTxt)

,text,label
0,【レンタル】振袖レンタル（2月〜12月）〔6S713 白地〕〔結婚式〕〔成人式〕〔結納〕〔卒業式〕【レンタル着物】【振袖】〔貸衣装〕【フルセット】【小物一式つき】〔振り袖〕【往復送料無料】【新品足袋プレゼント】★白地に目の冴える赤ぼかし。およばれ パーティーなどにもアレンジを効かせて！※カレンダーの表示に少しお時間がかかります。表示までしばらくお待ちください。×印のところは申し訳ございませんが既にご予約済みとなっております。ご了承下さい。カレンダーの予約状況が反映されるまで時間がかかることがありますので、予約確定でない場合がございます。ショップからのメールにてご確認をお願い致します。モデル身長：163cm ※商品写真は撮影状況、お客様の表示環境等により実物と印象の異なる場合がございます。あらかじめご了承ください。 ※画像の小物類は、撮影用の為お付けする物と異なります ■付属品 振袖／袋帯／長襦袢（白半衿付き）／帯あげ／帯締め／草履／バッグ／肌着（ワンピースタイプ）／腰ひも 4本／和装ベルト／前板／後板／伊逹締め 2枚／帯枕／三重仮紐／重ね衿／衿芯／足袋（新品プレゼント） 振袖レンタル（2月〜12月）〔結婚式〕〔成人式〕〔結納〕〔卒業式〕【レンタル着物】【振袖】〔貸衣装〕【フルセット】【小物一式つき】〔振り袖〕【往復送料無料】【新品足袋プレゼント】★〔6s713 白地〕 白地に目の冴える赤ぼかし。およばれ パーティーなどにもアレンジを効かせて！ ■着物対応可能身長・サイズ 163cm位まで 13号まで ■色/柄：白地 赤薔薇ぼかし ■生地 着物：正絹 【ご用意して頂く物】 補正用タオル ※念の為、着付けされる美容院様にご確認ください。,Rental
1,"【店内全品ポイント10倍！11/5 23:59まで】七五三 着物 レンタル 7歳 フルセット アンティーク 正絹 【七歳】おはりばこの七五三レンタル衣裳♪正絹アンティーク着物で七五三！！【No.n07】【花輪乱菊】【桃色紋錦紗地花丸紋尽四つ身】【巾着＆はこせこ付き】fy16REN07おはりばこの七五三レンタル衣裳・レンタル衣装♪正絹アンティーク着物で七五三！ 桃色紋錦紗地花丸紋尽四つ身 鮮やかな桃色の地色に、菊がたくさん描かれた大きな丸紋が染められています。 勢いのあるリアルな乱菊と、文様化されたねじ梅の対比が面白いですね。 地色はとってもガーリーなピンク色。差し色も朱色や赤など、女の子が大好きなお色ばかり♪ 着るだけで昔のお嬢様になれそうな、かわいいお着物です。 ※帯まわりのコーディネイト（帯、帯揚げ、帯締め、しごき）について、 予約状況により、画像のものと異なる場合がございますが、 あらかじめご了承ください。 桃色紋錦紗地花丸紋尽四つ身 商品情報 サイズ 身丈:約122cm 裄丈:約51cm 適応身長:105〜117cm ※肩上げをした状態でお送りします。腰上げはしておりません。 ※アンティーク着物のため、現代のお着物とは仕立て・サイズが異なる場合がございます。 着物とお子様の裄丈の寸法を必ずご確認ください。&gt;&gt;&gt;【裄丈の測り方はこちら】 素材 表地:正絹 裏地:正絹 製造 国産 戦前のアンティーク着物 配送方法 宅急便（ゆうパック） 注意事項 商品代金の支払は、カード決済のみとなります。 銀行振込・代金引換はご利用いただけませんのご了承ください。 ■「安心保険」ついて 七五三の晴れの日の衣裳をご検討頂きありがとうございます。 おはりばこでは、商品代金の5％の料金で保険が掛けられます！！ レンタルのご利用規約にも記載させて頂いておりますように、損傷し修復が必要となる場合は修復代金に相当する金額を弁償して頂くこととなっております。 例えば・・・ ・車の汚れが着物についてしまった。 ・子供が着物の裾を踏んで汚してしまった。 ・子供用の扇子を開け閉めして壊してしまった。（等 そんなときに安心できるのが、『安心保険サービス』でございます。 お衣裳の1セットにつき、レンタル代金の5％のご料金をお支払い頂けましたら、修理代金は一切発生致しません。 安心して当日をお過ごし頂きます為に、レンタルご利用のお客様すべてに、ご加入をお勧めさせて頂いております。 ■安心保険適用外 事項 下記の事項が発生致しました場合には、保険適用外となります。 ※使用不可能となる汚れ、破れ等の破損した場合。 ※飲食による汚れ。 ※タバコの焼け焦げ。 ※レンタル商品を紛失された場合。 （等 ■レンタルお申込み期間について ご注文日より最短10日以降、最長約半年までの期間内のみ受付いたしております。 ■レンタルには、当店での髪飾りまたは七五三商品の購入が必要です。 レンタル着物をご購入とご一緒に当店おはりばこの髪飾りまたは七五三商品を 5,000円以上ご購入くださいませ。 ※レンタル着物ご注文時点より、 過去半年以内に当店の髪飾りをご購入のお客様も対象となり、レンタルが可能です。 レンタル着物購入後に、髪飾りを選んで購入希望のお客様は、 レンタル利用日までご注文くださいませ。 髪飾りについてコーディネイト・ご相談などはお気軽にご連絡くださいませ。 ■髪飾りをご検討中のお客様へ 毎年10月頃から完売商品も出てまいりますので、髪飾りをすでに決めている場合は、先に楽天市場店または、実店舗、催事出店などでご購入いただくことをおすすめいたします。 また、七五三髪飾りは5月末〜11月末までの製作となっております。前撮り・後撮りの場合、在庫がない場合もございますが、ご了承ください。 ■おはりばこの七歳用の帯は作り帯ではありません！ 当店の七歳用の帯は、大人用の高級丸帯を子供用に仕立て直した帯です。 振袖の着付けなどと同じように、帯結びのできる着付師さんが必要です。 必ず写真館や美容室にご確認の上、ご利用ください。 ※三歳用帯コーデの帯は作り帯です。 ■アンティーク着物には、時代の汚れや傷みがあります。 おはりばこが扱う七五三衣裳は、全て戦前に作られたアンティーク着物です。 70年以上前に作られたお着物ですので、 時代相応の汚れ・やけ・色あせ・すれ・傷みが必ずあります。 もちろん、晴れ着として着用いただけるレベルのものばかりを取り揃えておりますが、 完全な状態のものは一着もありません。 汚れなどが気になる方は、ご利用をご遠慮いただくようおねがいいたします。 ■着用しての外出・お詣りについて 7月、8月のご利用に関して、汗じみ防止のため、着用しての外出やお詣りは不可となっています。室内での撮影のみでのご利用をお願いいたします。 その他の時期に関しては、室内での撮影はもちろん外出・お詣りは可能でございます。",Rental
2,【レンタル】【留袖 レンタル】レンタル留袖 mito_4 北256 グリーン のし 正絹|母親 黒留袖 フルセット マイセレクト 留袖レンタル レンタル着物 草履バッグセット 黒留袖レンタル 柄 結婚式 着物セット 黒留 帯締め 黒留め袖 貸衣装 お宮参り おすすめ レディース厳選された留袖 対応身長 155cmから165cm位 洋服号数 7〜13号(ヒップサイズ97cm以下) 当店ではアップグレードの表地：織りの日本製ブランド草履バッグがございます。↓ 商品詳細 【商品番号】 mito_4：北256 グリーン のし鶸色のぼかしで鬘帯文に洋花を染め抜き大輪の菊を金彩でアクセントにしたモダン感覚の留袖です。 【素材】 表地：正絹100% 裏地：比翼仕立合成繊維 【サイズ】 裄70cm・着丈165cm・袖丈49cm サイズ表は*こちらをクリック* してください。 【対応身長】 155cmから165cm位 ／ 洋服サイズ 7〜13号位まで 【注意事項】 ※商品の色目につきましては、お客様のご覧になられますパソコン機器及びモニタなどの違いにより実際の商品素材の色と相違する場合が御座います。予めご了承下さい。 【注意事項2】 ※帯に関しましては、商品の出荷状況などにより同じ物をご用意させて頂くお約束ができません。同じような帯のイメージでコーディネートさせて頂きますので、ご了承の程、よろしくお願い致します。 【コーディネート】 帯柄、帯揚、帯〆、伊達衿、半衿、草履、バックは当店プロスタッフがお着物にあったものをコーディネートさせて頂きます。 ご要望などのあるお客様は備考欄に具体的にご記入ください。,Rental
3,"【レンタル】[東京礼服センター発送商品][フルセット]喪服 レンタル 礼服 レンタル〔K032〕桂由美 レディース ブラックフォーマル アンサンブル ワンピース レンタル[お通夜][お葬式]{9号}{11号}[女性 礼服][レディース]【fy16REN07】「礼装バッグ」「靴」「ネックレス」すべてそろったフルセットレンタル 「礼装用ストッキング」プレゼント♪ 喪服 礼服 レンタル クリーニング不要◆ご利用日の2日前にお届けします。 （ご注文いただいた日時によっては2日前にお届けできない場合がございます） ◆お急ぎの際はご利用日に間に合うようお届けします。 東京・大阪翌日午前中にお届けできます。 本州全域翌日のお届けが可能です（時間帯についてはお問い合わせください）。 ◆ご利用日の翌日に返却（発送）をお願いいたします。 ◆シーズン・土日祝日は注文が大変混みあいます。 お早目のご注文をお願いいたします。 ご注文いただきましたサイズがすべてご予約済の場合はメールもしくはお電話にてお知らせいたし

CPU times: user 357 ms, sys: 173 ms, total: 529 ms
Wall time: 655 ms


In [43]:
%%time
# extract nouns and verbs (feature selecting)
mecab = MeCab.Tagger('-d /usr/local/lib/mecab/dic/mecab-ipadic-neologd')

def extractWords(doc):
    keywords = []
    parsedStr = mecab.parse(doc)
    for chunk in parsedStr.splitlines()[:-1]:
        (surface, feature) = chunk.split('\t')
        if feature.startswith('名詞') or feature.startswith('動詞'):
            lemma = feature.split(',')[6]
            if lemma != '*':
                keywords.append(lemma)
            else:
                keywords.append(surface)
    return keywords


bows = []
txtList = list(allTxt['text'])
for row in tqdm_notebook(txtList):
    bow = extractWords(row)
    bows.append(bow)
display(pd.DataFrame(bows))

,0,1,2,3,4,5,6,7,8,9,...,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982
0,レンタル,振袖,レンタル,2月,12月,6,S,713,白地,結婚式,...,None,None,None,None,None,None,None,None,None,None
1,店内,全品,ポイント,10倍,11,5,23,59,七五三,着物,...,None,None,None,None,None,None,None,None,None,None
2,レンタル,留袖,レンタル,レンタル,留袖,mito,4,北,256,グリーン,...,None,None,None,None,None,None,None,None,None,None
3,レンタル,東京,礼服,センター,発送,商品,フルセット,喪服,レンタル,礼服,...,None,None,None,None,None,None,None,None,None,None
4,レンタル,振袖,レンタル,セット,R1,674,赤,平安朝,御簾,松牡丹,...,None,None,None,None,None,None,None,None,None,None
5,レンタル,産着,紺,ぼかし,兜,MD,2004,ベビー,帽子,セット,...,None,None,None,None,None,None,None,None,None,None
6,得,レンタル,1,ヶ月,Maxi-Cosi,クイニーザップ,Maxicosi,Quinny,Zapp,ベビーカー,...,None,None,None,None,None,None,None,None,None,None
7,往復,送料,無料,七五三,レンタル,七五三,着物,男の子,5歳,男児,...,None,None,None,None,None,None,None,None,None,None
8,得,レンタル,3,ヶ月,コンビ,White Label,メチャカル,ハンディ,エッグ,ショック,...,None,None,None,None,None,None,None,None,None,None
9,訪問着,レンタル,フルセット,正絹,着物,結婚式,卒業式,入学式,七五三,身長,...,None,None,None,None,None,None,None,None,None,None


CPU times: user 6min 53s, sys: 58.1 s, total: 7min 51s
Wall time: 11min 5s


In [37]:
%%time
# make all words set
wordSet = set().union(*bows)

display(wordSet)

{'ぬける',
 'スパハイパースポーツサプリメント',
 '971985250628',
 'EXTREME',
 '身近',
 '014247',
 'シロエビ',
 '1136',
 '9710',
 'ピンクナナメレース',
 'フリンジラテンワンピース',
 '0.96kg',
 'Zune',
 '00433',
 '9332402019870',
 'カミニート',
 '童話',
 '72003',
 'MCP',
 'Look',
 '10月19日',
 '4杯',
 '979569242267',
 '975333401007',
 '4973603205645',
 '4977082206384',
 '大七宝',
 '01337',
 'タイドグラフ',
 'ELectricaL',
 'グレーウッドグレイン',
 'UMASHIMA',
 'テスコガビー',
 'サンゴベルベットソール',
 '美瑛',
 'GP32',
 'コンポジション',
 '石村',
 'IND',
 'ルミカ',
 'ウェーブパーマ',
 '対抗',
 '4976555928815',
 '激励',
 '3.7kg',
 '8117940143086',
 'シユウエイシヤ',
 '1850年代',
 'シーズ・ア・レディー',
 'Umo',
 'プラウザ',
 '方形',
 'フレキシボール',
 '4078',
 '013479',
 'リーミー',
 'お湯',
 '4994527852005',
 '5200303830570',
 'バイカラーコンビワンピース',
 '仮想世界',
 'ラッキーブルー',
 '冷却',
 '845m',
 '979508846099',
 '977389661237',
 '004035',
 '白羽毛',
 '902430755559',
 'herramientas',
 '4961503654243',
 '906901147317',
 '430615',
 'Fnキー',
 '7403',
 '013808',
 '21740',
 '512134',
 '5073594',
 'ハイドン',
 'ポイントガード',
 '9784344019898',
 'シークレットシェイプスリッパ',
 'ダイヤスキンシー

CPU times: user 4.79 s, sys: 2.08 s, total: 6.87 s
Wall time: 8.32 s


In [ ]:
%%time
wordDicts = []
wordDictInit = dict.fromkeys(wordSet, 0)

for bow in tqdm_notebook(bows):
    wordDict = wordDictInit
    for word in bow:
        wordDict[word] += 1
    wordDicts.append(wordDict)

display(pd.DataFrame.from_dict(wordDicts))